## Imports and Setup

In [1]:
import pandas as pd
import numpy as np
import nltk.tokenize
import itertools
import datetime
import torch

from sklearn.model_selection import train_test_split

from pathlib import Path

from fastai import *
from fastai.text import *

from copy import copy, deepcopy
from enum import Enum

### Model definitions
Define this notebooks "initializer model" (pretrained parameters and itos mapping) and the output model as well.

In [9]:
model_name = '4.2-LM-108k-lines-genre-song_title'
MODEL_PATH = Path(f'../data/models/{model_name}')
MODEL_PATH.mkdir(exist_ok=True)

init_model_name = '4.1-LM-108k-lines'
INIT_MODEL_PATH = Path(f'../data/models/{init_model_name}')
INIT_MODEL_PATH.mkdir(exist_ok=True)

### Tokenization
To create the model's tokens with the correct train-test split, run the code below. Only needed once on the notebook's first ever run.

In [3]:
MODEL_FIRST_RUN = False

In [4]:
def tokenize_lyrics_with_tags(lyrics, line_num=True, genre=False, song_title=False):
    tk = nltk.tokenize.LineTokenizer(blanklines='keep')
    tokd = tk.tokenize(lyrics)
    
    re_tk = nltk.tokenize.RegexpTokenizer(r'\[[^\]]+\]|\w+|[\d\.,]+|\S+',
                                          discard_empty=False)
    re_tokd = re_tk.tokenize_sents(tokd)

    if genre:
        [s.insert(0, f'xGENRE') for s in re_tokd] # insert start token for each line

    elif song_title:
        [s.insert(0, f'xTITLE') for s in re_tokd] # insert start token for each line

    else:
        if line_num:
            [s.insert(0, f'xBOL {line_num+1}') for line_num, s in enumerate(re_tokd)] # insert start token for each line
        else:
            [s.insert(0, f'xBOL') for s in re_tokd] # insert start token for each line

        [s.append('xEOL') for s in re_tokd] # append end token for each line

    flat = list(itertools.chain(*re_tokd))
    # lower case and de-space
    flat = [w.lower().replace(' ', '-') for w in flat]
    return flat

In [5]:
def process_tokens(model_path):
    '''
    500k link: https://storage.googleapis.com/capstone-deep-lyrics/lyrics-500k.csv
    108k link: https://storage.googleapis.com/w210-capstone/data/lyrics-valid.csv
    '''
    model_path = Path(MODEL_PATH)
    model_path.mkdir(exist_ok=True)

    small_corpus_url = 'https://storage.googleapis.com/w210-capstone/data/lyrics-valid.csv'
    audio_url = 'https://storage.googleapis.com/w210-capstone/data/msd-aggregate.csv'
    tag_url = 'https://storage.googleapis.com/w210-capstone/data/lyrics_tags.csv'

    # load scraped data
    df = pd.read_csv(small_corpus_url,
                     header=None, escapechar='\\',
                     names=['msd_id', 'lyrics'])

    # only keep lyrics with length < 5000
    df = df[df.lyrics.str.len() < 5000]

    # bring in audio features
    df_audio = pd.read_csv(audio_url)
    df_audio.rename(columns={'track_id': 'msd_id'}, inplace=True)

    # bring in tags
    df_tags = pd.read_csv(tag_url, index_col=0)
    df_tags.drop(columns=['lyrics'], inplace=True)

    # match audio to keys from lyrics
    df_audio = pd.merge(df, df_audio, how='inner', on='msd_id')

    # mat
    df_all = pd.merge(df_audio, df_tags, how='inner', on='msd_id')
    df = df_all[['tag1', 'tag2', 'title', 'lyrics']]

    print('Tokenizing...')
    lyrics = df.lyrics.apply(tokenize_lyrics_with_tags, line_num=True)
    genre = df.tag1.astype(str).apply(tokenize_lyrics_with_tags, line_num=False, genre=True)
    song_title = df.title.apply(tokenize_lyrics_with_tags, line_num=False, song_title=True)

    full_song = (['xbos'] + genre + song_title + lyrics + ['xeos'])

    # split train/test
    df_train, df_test = train_test_split(full_song, test_size=0.2, random_state=2018)

    # tokens
    print('Saving...')
    tokens = np.array(df_train)
    np.save(model_path/'train_tok.npy', tokens)

    tokens = np.array(df_test)
    np.save(model_path/'valid_tok.npy', tokens)
    print('Done.')

In [6]:
if MODEL_FIRST_RUN:
    process_tokens(MODEL_PATH)

## Load Data

Now that we've created the tokens, let's load them into a `DataBunch` to train our LM further or generate text with a pre-trained LM.

In [7]:
if MODEL_FIRST_RUN:
    data_lm = TextLMDataBunch.from_tokens(MODEL_PATH,
                                          bs=128,
                                          max_vocab=20000)

In [11]:
if not MODEL_FIRST_RUN:
    data_lm = TextLMDataBunch.from_id_files(MODEL_PATH/'tmp')
    data_lm.path = MODEL_PATH

In [12]:
data_lm.train_ds.vocab_size

20002

## Model setup

In [13]:
GPU = True
DOWNLOAD_INIT_MODEL = False

In [14]:
learn = RNNLearner.language_model(data_lm,
                                  drop_mult=0.5)

In [15]:
# weights_url = 'https://storage.googleapis.com/capstone-deep-lyrics/3.2-ULMFiT-108k_best.pth'
# itos_url = 'https://storage.googleapis.com/capstone-deep-lyrics/3.2-ULMFiT-108k_best.pth'

# if DOWNLOAD_INIT_MODEL:
#     Path(MODEL_PATH/'models').mkdir(exist_ok=True)
#     download_url(weights_url, MODEL_PATH/f'models/{model_name}_best.pth', overwrite=False)
#     download_url(weights_url, MODEL_PATH/f'models/{model_name}_best.pth', overwrite=False)

In [16]:
if DOWNLOAD_INIT_MODEL:
    untar_data('https://s3.amazonaws.com/fast-ai-modelzoo/wt103',
               dest=MODEL_PATH/'models'/f'{init_model_name}')
    
    # trained weights
    with open(MODEL_PATH/'models'/f'{model_name}_best.pth', 'wb') as f:
        res = requests.get(f'https://storage.googleapis.com/w210-capstone/models/{model_name}_best.pth')
        f.write(res.content)

**Create a folder in this model's directory with the name of the initializer model. In that folder, copy a .pth file in the form lstm_<init_model_name>.pth and a .pkl file in the form itos_<init_model_name>.pkl' in order to transfer weights and tokens correctly from the initializer model**

In [17]:
if MODEL_FIRST_RUN:
    learn.load_pretrained(MODEL_PATH/f'{init_model_name}/lstm_{init_model_name}.pth', 
                          MODEL_PATH/f'{init_model_name}/itos_{init_model_name}.pkl')

In [18]:
def cpu_load(self, name:PathOrStr):
    """Load model onto CPU that was trained on a GPU `name` from `self.model_dir`.
       We need these because the fastai load function doesn't allow for a remapping of the storage location."""
    self.model.load_state_dict(torch.load(self.path/self.model_dir/f'{name}.pth', map_location=lambda storage, loc: storage))

setattr(RNNLearner, 'cpu_load', cpu_load) #monkey patch onto our RNNLearner

In [19]:
if not MODEL_FIRST_RUN:
    if not GPU:
        learn.cpu_load(f'{model_name}_best')
    else:
        learn.load(f'{model_name}_best')

## Training

In [20]:
TRAIN = False

In [21]:
@dataclass
class SaveModel(LearnerCallback):
    """Save Latest Model"""
    def __init__(self, learn:Learner, model_name='saved_model'):
        super().__init__(learn)
        self.model_name = model_name
        self.model_date = datetime.datetime.now().strftime('%Y-%m-%d-%H-%M-%S')
        self.best_loss = None
        self.perplexity = []
        
    def on_epoch_end(self, epoch:int, metrics, last_metrics, **kwargs):
        loss, *_ = last_metrics
        perp = np.exp(loss)
        self.perplexity.append(perp)
        if self.best_loss == None or loss < self.best_loss:
            self.best_loss = loss
            self.learn.save(f'{self.model_name}_best')
        return False
    
    def on_train_end(self, epoch:int, **kwargs):
        self.learn.save(f'{self.model_name}_last')

In [22]:
save_callback = SaveModel(learn, model_name=f'{model_name}')

In [23]:
if TRAIN:
    learn.fit_one_cycle(1, 1e-2)

In [24]:
if TRAIN:
    learn.unfreeze()
    learn.fit(10, 1e-3, callbacks=[save_callback])

In [25]:
print("best validation loss: ", learn.save_model.best_loss)

best validation loss:  None


In [26]:
#learn.save_encoder(f'{model_name}_encoder')

### Learning Loss

In [27]:
if TRAIN:
    learn.recorder.plot_losses()

### Model Output

In [31]:
if TRAIN: pass
MODEL_OUTPUT_NAME = MODEL_PATH/'models'/f'{model_name}_best.pth'
CLOUD_STORAGE = f'gs://w210-capstone/models/{model_name}_best.pth'

In [32]:
!gsutil cp $MODEL_OUTPUT_NAME $CLOUD_STORAGE

Copying file://../data/models/4.2-LM-108k-lines-genre-song_title/models/4.2-LM-108k-lines-genre-song_title_best.pth [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/150.5 MiB.                                    


In [28]:
ARCHITECTURE_PATH = MODEL_PATH/'models'/f'{model_name}_architecture.pkl'
ARCHITECTURE_STORAGE = f'gs://w210-capstone/models/{model_name}_architecture.pkl'
ITOS_PATH = MODEL_PATH/'tmp'/'itos.pkl'
ITOS_STORAGE = f'gs://w210-capstone/models/{model_name}_itos.pkl'

model_dump = learn.model
with open(ARCHITECTURE_PATH, 'wb') as f:
    pickle.dump(model_dump, f)

In [29]:
!gsutil cp $ARCHITECTURE_PATH $ARCHITECTURE_STORAGE

Copying file://../data/models/4.2-LM-108k-lines-genre-song_title/models/4.2-LM-108k-lines-genre-song_title_architecture.pkl [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

|
Operation completed over 1 objects/381.8 MiB.                                    


In [30]:
!gsutil cp $ITOS_PATH $ITOS_STORAGE

Copying file://../data/models/4.2-LM-108k-lines-genre-song_title/tmp/itos.pkl [Content-Type=application/octet-stream]...
/ [1 files][310.2 KiB/310.2 KiB]                                                
Operation completed over 1 objects/310.2 KiB.                                    


## Text Generation

In [33]:
%load_ext autoreload
%autoreload 2

In [34]:
from src.nlp.generate_lyrics import DeepLyric

In [35]:
deep_lyric = DeepLyric(learn.model, weights=None, itos=data_lm.train_ds.vocab.itos)

In [ ]:
deep_lyric.

In [55]:
deep_lyric.generate_text(seed_text='xbos xgenre country', 
                         verbose=1, context_length=100, beam_width=3,
                         max_len=50, top_k=5, temperature=1.45,
                         GPU=True)

Generating word: 1 / 50
Generating word: 2 / 50
Generating word: 3 / 50
Generating word: 4 / 50
Generating word: 5 / 50
Generating word: 6 / 50
Generating word: 7 / 50
Generating word: 8 / 50
Generating word: 9 / 50
Generating word: 10 / 50
Generating word: 11 / 50
Generating word: 12 / 50
Generating word: 13 / 50
Generating word: 14 / 50
Generating word: 15 / 50
Generating word: 16 / 50
Generating word: 17 / 50
Generating word: 18 / 50
Generating word: 19 / 50
Generating word: 20 / 50
Generating word: 21 / 50
Generating word: 22 / 50
Generating word: 23 / 50
Generating word: 24 / 50
Generating word: 25 / 50
Generating word: 26 / 50
Generating word: 27 / 50
Generating word: 28 / 50
Generating word: 29 / 50
Generating word: 30 / 50
Generating word: 31 / 50
Generating word: 32 / 50
Generating word: 33 / 50
Generating word: 34 / 50
Generating word: 35 / 50
Generating word: 36 / 50
Generating word: 37 / 50
Generating word: 38 / 50
Generating word: 39 / 50
Generating word: 40 / 50
Generatin

In [56]:
deep_lyric.print_lyrics()

SONG START
 genre: country 
 title: the kiss of gold 

xbol-1 when i 'm with you 
 there 's so much out there for me 
 here and now , my dear 
 there is nothing more left to do 
 
 when my whole life through 
 i 

In [57]:
deep_lyric.save_lyrics_to_file('test.txt')

In [59]:
for i in range(10):
    deep_lyric.generate_text(max_len=20)
    deep_lyric.save_lyrics_to_file(f'test{i}.txt', GPU=True)

Generating word: 1 / 20


RuntimeError: Expected object of backend CUDA but got backend CPU for argument #3 'index'